##Tải về tập dữ liệu flickr8k đã clean

In [1]:
!git clone https://github.com/DoanNgocToan/clean_data_flickr8k

Cloning into 'clean_data_flickr8k'...
remote: Enumerating objects: 8123, done.
remote: Total 8123 (delta 0), reused 0 (delta 0), pack-reused 8123 (from 2)
Receiving objects: 100% (8123/8123), 1.03 GiB | 37.63 MiB/s, done.
Resolving deltas: 100% (15/15), done.
Updating files: 100% (8095/8095), done.


In [2]:
!ls /content/clean_data_flickr8k/

captions.txt  Images  README.md  vocabulary.txt


##Đọc file caption

In [4]:
import os
from collections import defaultdict
from IPython.display import display, Image

dataset_path = '/content/clean_data_flickr8k'
caption_path = os.path.join(dataset_path, 'captions.txt')
image_dirs = os.path.join(dataset_path, 'Images')
# Create a directory to save image paths and captions
image_caption = defaultdict(list)
image_paths = set()
# Các thông số cần thiết
caption_counts = 0 # Số lượng caption
# Read the text file

if os.path.exists(caption_path):
  with open(caption_path, 'r') as f:
    next(f) # Bỏ qua tiêu đề 'image,caption'
    for line in f:
      # Split only at the first comma to correctly separate image and caption
      dir_cap = line.strip().split(',', 1)
      image_path = os.path.join(image_dirs, dir_cap[0])
      image_paths.add(image_path)
      if os.path.exists(image_path):
        image_caption[image_path].append(dir_cap[1])

##Tiền xử lý dữ liệu



In [5]:
import re
from collections import defaultdict

# Khởi tạo lại các biến cho việc làm sạch
cleaned_vocab_set = set()
cleaned_max_lengths = 0
cleaned_image_caption = defaultdict(list)

# Thực hiện làm sạch dữ liệu từ image_caption và image_paths đã có
for path in image_paths:
    if path not in image_caption:
        continue
    for caption in image_caption[path]:
        cleaned_words_for_caption = []
        raw_words = caption.split()
        for word in raw_words:
          # Loại bỏ các dấu câu thường gặp trong captions
            cleaned_word = word.strip("'.,!#():;?&- ").strip('"').lower()
            if cleaned_word:
                cleaned_words_for_caption.append(cleaned_word)

        # Gộp các từ đã làm sạch lại thành một caption và thêm vào cleaned_image_caption
        cleaned_caption_string = ' '.join(cleaned_words_for_caption)
        if cleaned_caption_string: # Only add non-empty cleaned captions
            cleaned_image_caption[path].append(cleaned_caption_string)

        # Update max_lengths based on the length of cleaned words
        if cleaned_max_lengths < len(cleaned_words_for_caption):
           cleaned_max_lengths = len(cleaned_words_for_caption)

        # Update cleaned_vocab_set with cleaned words
        cleaned_vocab_set.update(cleaned_words_for_caption)

# In thống kê sau khi làm sạch nhẹ
print("\n--- Thống kê sau khi làm sạch nhẹ (loại bỏ \".,'!#():;?) ---")
print(f"Kích thước từ vựng sau làm sạch: {len(cleaned_vocab_set)}")
print(f"Độ dài caption dài nhất sau làm sạch: {cleaned_max_lengths}")

# --- Tìm và in các từ chứa ký tự không phải chữ cái hoặc số ---
print("\nCác từ trong từ vựng SAU LÀM SẠCH chứa ký tự không phải chữ cái hoặc số:")
non_alphanum_words_cleaned = []
for word in sorted(list(cleaned_vocab_set)): # Sort for consistent output
    # Check if the word contains any character that is NOT a letter or a digit
    if re.search(r'[^a-z0-9-]', word): # Dấu '-' cuối là để loại các từ ghép có dấu gạch nối
        non_alphanum_words_cleaned.append(word)

if non_alphanum_words_cleaned:
    for word in non_alphanum_words_cleaned:
        print(word)
else:
    print("Không có từ nào trong từ vựng sau làm sạch chứa ký tự không phải chữ cái hoặc số.")


--- Thống kê sau khi làm sạch nhẹ (loại bỏ ".,'!#():;?) ---
Kích thước từ vựng sau làm sạch: 8455
Độ dài caption dài nhất sau làm sạch: 36

Các từ trong từ vựng SAU LÀM SẠCH chứa ký tự không phải chữ cái hoặc số:
at&t
avrovulcan.com
d.c
n't
o'clock
r.v
s.c.u.b.a
statefarm.com


## Padding và Đánh Chỉ Số Cho Nhãn

In [6]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# ============================
# 1. Thêm các token đặc biệt vào từ vựng
# ============================
# Tạo một bản sao của tập từ vựng đã làm sạch
vocabulary_for_tokenizer = set(cleaned_vocab_set)

# Thêm các token đặc biệt
vocabulary_for_tokenizer.add('<start>')
vocabulary_for_tokenizer.add('<end>')
vocabulary_for_tokenizer.add('<pad>')

# Chuyển set thành list để truyền vào Tokenizer
vocabulary_list = sorted(list(vocabulary_for_tokenizer))

# ============================
# 2. Khởi tạo Tokenizer và xây dựng chỉ mục từ
# ============================
# num_words: None (giữ tất cả các từ), oov_token: đại diện cho các từ không có trong từ vựng
tokenizer = Tokenizer(num_words=None, oov_token='<unk>')

# Fit trên tập các từ vựng đã bao gồm các token đặc biệt
tokenizer.fit_on_texts(vocabulary_list)

# Đảm bảo các token đặc biệt có chỉ mục cố định và là các chỉ mục thấp nhất
# (thường tokenizer tự làm điều này nếu chúng được thêm vào và fit_on_texts)
# Tuy nhiên, nếu muốn đảm bảo, có thể chỉnh sửa word_index sau khi fit.
# Ví dụ: nếu '<pad>' thường được gán chỉ mục 0 bởi pad_sequences, ta có thể bỏ qua.
# Keras pad_sequences mặc định padding_value=0, nên ta cần đảm bảo <pad> = 0 hoặc điều chỉnh padding_value.
# Ở đây, ta sẽ giả định tokenizer gán chỉ mục từ 1 trở đi, và pad_sequences sẽ dùng 0 cho padding.

# Gán lại chỉ mục cho các token đặc biệt nếu cần để đảm bảo tính nhất quán
# Đây là một bước tùy chọn, tùy thuộc vào yêu cầu cụ thể về chỉ mục của các token đặc biệt.
# Trong nhiều trường hợp, việc để tokenizer tự động gán là đủ.
# Để đảm bảo <pad> là 0, chúng ta cần xây dựng word_index thủ công hoặc điều chỉnh sau.
# Cách đơn giản nhất là để pad_sequences thêm 0 và sử dụng các chỉ mục khác cho các từ.
# Tokenizer tự động gán 1 cho từ phổ biến nhất, 2 cho từ phổ biến thứ 2, v.v.
# OOV_token sẽ nhận chỉ mục cuối cùng (num_words+1 nếu num_words được set)

# Để đảm bảo <pad> là 0, chúng ta sẽ xây dựng lại `word_index` một cách thủ công hoặc điều chỉnh nó.
word_index = tokenizer.word_index
# Tạo word_index mới với <pad> = 0, <start> = 1, <end> = 2, <unk> = 3 (hoặc các giá trị khác)
# Sau đó gán các từ còn lại.

# Reset tokenizer để xây dựng thủ công
tokenizer = Tokenizer(num_words=None, oov_token='<unk>')

# Tạo từ điển chỉ mục thủ công để đảm bảo vị trí các token đặc biệt
# Note: Keras Tokenizer mặc định các chỉ mục từ 1 trở đi. 0 được giữ cho padding.
# Nếu bạn muốn <pad> có chỉ mục 0, thì các từ khác sẽ bắt đầu từ 1.
# Chúng ta sẽ để Keras làm phần lớn, chỉ đảm bảo các token đặc biệt được xử lý.
# Thêm các token đặc biệt vào đầu danh sách từ vựng để chúng có chỉ mục thấp.
# Nhưng tốt nhất là để Tokenizer tự học và sau đó điều chỉnh (hoặc sử dụng một mapping khác).

# Để đơn giản, ta sẽ để tokenizer tạo word_index và sau đó điều chỉnh index_word và word_index
# nếu muốn chỉ mục 0 là <pad>. Hoặc đơn giản hơn, thêm nó vào tập từ vựng và để tokenizer gán chỉ mục.
# Tokenizer sẽ gán chỉ mục từ 1 trở đi cho các từ, giữ 0 cho padding.
# OOV token sẽ có chỉ mục dựa trên số lượng từ. Để đơn giản, ta sẽ thêm các token này vào trước khi fit.

# Cách phổ biến là tạo thủ công một số chỉ mục đầu tiên cho các token đặc biệt
# và sau đó thêm các từ còn lại.

# Danh sách các từ có thể xuất hiện trong captions, bao gồm các token đặc biệt.
all_words = ['<pad>', '<start>', '<end>', '<unk>'] + sorted(list(cleaned_vocab_set))

# Khởi tạo lại tokenizer
tokenizer = Tokenizer(filters='', lower=True, split=' ', oov_token='<unk>')
tokenizer.fit_on_texts(all_words)

# Điều chỉnh word_index và index_word để <pad> là 0, <start> là 1, <end> là 2, <unk> là 3
# Keras Tokenizer tự động bỏ qua 0 cho padding, nên chỉ mục sẽ bắt đầu từ 1.
# Để đảm bảo, chúng ta sẽ hoán đổi nếu cần hoặc gán thủ công.
# Mặc định, tokenizer sẽ gán chỉ mục 1 cho từ xuất hiện nhiều nhất.
# Để đảm bảo các token đặc biệt có chỉ mục thấp, ta có thể xây dựng thủ công.

# Xây dựng thủ công word_index và index_word
word_index = {'<pad>': 0, '<start>': 1, '<end>': 2, '<unk>': 3}
index_word = {0: '<pad>', 1: '<start>', 2: '<end>', 3: '<unk>'}
current_index = 4

for word in sorted(list(cleaned_vocab_set)):
    if word not in word_index:
        word_index[word] = current_index
        index_word[current_index] = word
        current_index += 1

# Cập nhật tokenizer với word_index và index_word đã được tạo thủ công
tokenizer.word_index = word_index
tokenizer.index_word = index_word

vocab_size = len(word_index)

print(f"Kích thước từ vựng (bao gồm các token đặc biệt): {vocab_size}")
print(f"Chỉ mục của <start>: {tokenizer.word_index['<start>']}")
print(f"Chỉ mục của <end>: {tokenizer.word_index['<end>']}")
print(f"Chỉ mục của <pad>: {tokenizer.word_index['<pad>']}")
print(f"Chỉ mục của <unk>: {tokenizer.word_index['<unk>']}")

# ============================
# 3. Chuyển đổi captions thành chuỗi số và padding
# ============================
# Tạo một danh sách các chuỗi caption đã được xử lý để truyền vào tokenizer
processed_captions = []

# Xác định độ dài tối đa của caption sau khi thêm <start> và <end>
# Nó sẽ là cleaned_max_lengths + 2 (cho <start> và <end>)
max_caption_length = cleaned_max_lengths + 2

for image_path, captions in cleaned_image_caption.items():
    for caption_text in captions:
        # Thêm token <start> và <end> vào caption
        processed_caption_with_tokens = '<start> ' + caption_text + ' <end>'
        processed_captions.append(processed_caption_with_tokens)

# Chuyển đổi các caption thành chuỗi số (sequences of integers)
encoded_sequences = tokenizer.texts_to_sequences(processed_captions)

# Padding các chuỗi số để chúng có cùng độ dài tối đa
# Post-padding để các token <end> không bị mất
padded_sequences = pad_sequences(encoded_sequences, maxlen=max_caption_length, padding='post', value=tokenizer.word_index['<pad>'])

print(f"\nSố lượng captions đã được mã hóa: {len(padded_sequences)}")
print(f"Độ dài caption tối đa (bao gồm token đặc biệt và padding): {max_caption_length}")
print("\nVí dụ về một caption gốc:", processed_captions[0])
print("Ví dụ về một caption đã được mã hóa và padding (dạng số):")
print(padded_sequences[0])

# ============================
# 4. Tạo từ điển ánh xạ file ảnh với các chuỗi số của caption
# ============================
image_to_padded_sequences = {}
caption_index = 0

for image_path, captions in cleaned_image_caption.items():
    # Lấy ra các sequences đã được padding tương ứng với các captions của ảnh này
    num_captions_for_image = len(captions)
    image_to_padded_sequences[image_path] = padded_sequences[caption_index : caption_index + num_captions_for_image]
    caption_index += num_captions_for_image

print(f"\nVí dụ về captions đã được padding cho một ảnh bất kỳ: {list(image_to_padded_sequences.keys())[0]}")
print(image_to_padded_sequences[list(image_to_padded_sequences.keys())[0]])

# Lưu trữ max_caption_length và vocab_size cho các bước sau
# Đây là các thông số quan trọng cho việc xây dựng mô hình
MAX_CAPTION_LENGTH = max_caption_length
VOCAB_SIZE = vocab_size


Kích thước từ vựng (bao gồm các token đặc biệt): 8459
Chỉ mục của <start>: 1
Chỉ mục của <end>: 2
Chỉ mục của <pad>: 0
Chỉ mục của <unk>: 3

Số lượng captions đã được mã hóa: 40455
Độ dài caption tối đa (bao gồm token đặc biệt và padding): 38

Ví dụ về một caption gốc: <start> a bird flies across the water <end>
Ví dụ về một caption đã được mã hóa và padding (dạng số):
[   1   55  730 2823   87 7518 8128    2    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0]

Ví dụ về captions đã được padding cho một ảnh bất kỳ: /content/clean_data_flickr8k/Images/3325578605_afa7f662ec.jpg
[[   1   55  730 2823   87 7518 8128    2    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0]
 [   1   55 8241  730 2823 5000   55 4023    2    0    0    0    0    0
     0    0    0    0    0    0    0    0    0  

In [7]:
print(f"Kiểu dữ liệu của padded_sequences: {type(padded_sequences)}")
print(f"Kiểu dữ liệu của các phần tử trong padded_sequences: {padded_sequences.dtype}")

Kiểu dữ liệu của padded_sequences: <class 'numpy.ndarray'>
Kiểu dữ liệu của các phần tử trong padded_sequences: int32
